In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from PIL import Image,ImageOps
import cv2
from skimage import data, restoration
import glob

In [ ]:
(xx_train, y_train), (xx_test, y_test) =keras.datasets.fashion_mnist.load_data()
np.dtype(xx_test)

In [ ]:
path='../input/landscape-image-colorization/landscape Images/gray/*.jpg'

In [ ]:
data_path=glob.glob(path)

In [ ]:
cv_img = []
#dim=(128,128)
for img in data_path:
    n1= cv2.imread(img,0)
    cv_img.append(n1)

In [ ]:
plt.imshow(cv_img[1230],cmap='gray')

In [ ]:
cv_img4=cv_img
d=[]
for i in range(7129):
    if np.shape(cv_img4[i])==(150,150):
        d=cv_img4[i]
        cv_img4[i]=d[np.newaxis,:,:]
    else:
        cv_img4[i]=cv_img4[i-1]
np.shape(cv_img4)
cv_img5=cv_img4
cv_img6=np.squeeze(cv_img5)
np.shape(cv_img6)

In [ ]:
# for i in range(7129):
#     if not(np.shape(cv_img[i])[0]==150 or np.shape(cv_img[i])[1]==150):
#         cv_img[i]=cv_img[i-1]

In [ ]:
x_train,x_test=cv_img6[:6000],cv_img6[6001:]
X_train,X_test=np.array(x_train),np.array(x_test)
X_train,X_test=X_train/255.0,X_test/255.0
np.shape(X_train)
print(X_train[0])

In [ ]:
encoder=keras.models.Sequential([
    keras.layers.Reshape([150,150,1],input_shape=[150,150]),
    keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(128,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(64,kernel_size=(3,3),padding="same",activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2))
    
])

In [ ]:
decoder=keras.models.Sequential([
#     keras.layers.UpSampling2D(size=(2,2)),
#     keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
#     keras.layers.UpSampling2D(size=(2,2)),

#     keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
#     keras.layers.UpSampling2D(size=(2,2)),

#     keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same'),
    keras.layers.Conv2DTranspose(128,kernel_size=3,strides=2,padding="valid",activation="relu",input_shape=[18,18,64]),
    keras.layers.Conv2DTranspose(256,kernel_size=3,strides=2,padding="valid",activation="relu"),
    keras.layers.Conv2DTranspose(1,kernel_size=3,strides=2,padding="same",activation="sigmoid"),
    keras.layers.Reshape([150,150])
])

In [ ]:
conv_ae=keras.models.Sequential([encoder,decoder])

In [ ]:
conv_ae.compile(optimizer="adam",loss="binary_crossentropy")
encoder.summary()
decoder.summary()
conv_ae.summary()

In [ ]:
history=conv_ae.fit(X_train,X_train,epochs=100,batch_size=128,validation_data=(X_test, X_test))

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.savefig('conv_net-9x9x64.eps',format='eps')

In [ ]:
plt.imshow(X_test[20],cmap='gray')

In [ ]:
pred=conv_ae.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
plt.imshow(pred[20],cmap='gray')

In [ ]:
psf = np.ones((3,3)) / 9
# deconvolved, _ = restoration.unsupervised_wiener(pred[20], psf)
deconv_RL=restoration.richardson_lucy(pred[0], psf)#, num_iter=30)

In [ ]:
plt.imshow(deconvolved,cmap='gray')

In [ ]:
plt.imshow(deconv_RL,cmap='gray')

In [ ]:
for i in range(2):
    plt.rcParams["figure.figsize"] = (10,10)
    plt.subplot(1,2,i+1)
    plt.imshow(X_test[i+505],cmap='gray')
    plt.savefig('10.eps',dpi=300)
        

In [ ]:
for i in range(2):
    plt.rcParams["figure.figsize"] = (10,10)
    plt.subplot(1,2,i+1)
    plt.imshow(pred[i+505],cmap='gray')
    plt.savefig('10-recon.eps',dpi=300)

In [ ]:
from PIL import Image

In [ ]:
im=Image.open('../input/waterloo/camera.tif')
im2=im.resize((150,150))
im3=np.array(im2)/255.0
im3=im3[np.newaxis,:,:]
# plt.imshow(im3,cmap='gray')
# plt.savefig('bird-orig.eps',format='eps')
# plt.show()
np.shape(im3)

In [ ]:
pr=conv_ae.predict(im3)

In [ ]:
plt.imshow(pr[0],cmap='gray')
plt.savefig('camera-0.6.eps',format='eps')

In [ ]:
import shutil
shutil.make_archive('file', 'zip', './')

In [ ]:
conv_ae.save_model('mymodel--convAE')

In [ ]:
import tensorflow as tf

In [ ]:
from skimage import metrics

In [ ]:
psnr=metrics.peak_signal_noise_ratio(im3,pr[0])


In [ ]:
print(psnr)

In [ ]:
file='../input/waterloo/*.tif'
images=[Image.open(image) for image in glob.glob(file)]
type(images)
len(images)

In [ ]:
psnr=[]
ssim=[]
mse=[]
for i in range(len(images)):
    im4=images[i]
    im5=im4.resize((150,150))
    im6=np.array(im5)/255.0
    im6=im6[np.newaxis,:,:]
    predicted=conv_ae.predict(im6)
    im7=np.squeeze(im6)
    p=metrics.peak_signal_noise_ratio(im7,predicted[0])
    m=metrics.mean_squared_error(im7,predicted[0])
    s=metrics.structural_similarity(im7,predicted[0])
    psnr.append(p)
    mse.append(m)
    ssim.append(s)

In [ ]:
psnr

In [ ]:
ssim

In [ ]:

mse

In [ ]:
import pandas as pd
df_psnr = pd.DataFrame(psnr)
df_mse = pd.DataFrame(mse)
df_ssim = pd.DataFrame(ssim)
df_psnr.to_csv('psnr-0.6.csv', index=False)
df_mse.to_csv('mse-0.6.csv', index=False)
df_ssim.to_csv('ssim-0.6.csv', index=False)

In [ ]:
import seaborn as sn
data=[[11.1973249,25.83335307,27.72223755],[0.088118527,0.004403061,0.003516231],[0.445937632,0.849040522,0.873436445]]
dat=np.array(data)
plt.figure(figsize=(8,8))
sn.set_theme(style="whitegrid")
sn.barplot(x=[0.2,0.4,0.6],y=dat[0],palette="Blues_d")
plt.xlabel('Compression Ratio')
plt.ylabel('PSNR')
plt.savefig('PSNR-1.eps',dpi=300)